In [1]:
#Execute them for first to install
!pip install xgboost --upgrade
!pip install yellowbrick
!pip install feature-engine

Requirement already up-to-date: xgboost in /usr/local/lib/python3.6/dist-packages (1.1.1)


### Problem Statement
> A retail company “ABC Private Limited” wants to understand the customer purchase behaviour (specifically, purchase amount) against various products of different categories. They have shared purchase summary of various customers for selected high volume products from last month.
The data set also contains customer demographics (age, gender, marital status, city_type, stay_in_current_city), product details (product_id and product category) and Total purchase_amount from last month.
Now, they want to build a model to predict the purchase amount of customer against various products which will help them to create personalized offer for customers against different products.

|Variable|Definition|
|:---- |:----
|User_ID|User ID|
|Product_ID|Product ID|
|Gender|Sex of User|
|Age|Age in bins|
|Occupation|Occupation (Masked)|
|City_Category|Category of the City (A,B,C)|
|Stay_In_Current_City_Years|Number of years stay in current city|
|Marital_Status|Marital Status|
|Product_Category_1|Product Category (Masked)|
|Product_Category_2|Product may belongs to other category also (Masked)|
|Product_Category_3|Product may belongs to other category also (Masked)|
|Purchase|Purchase Amount (Target Variable)|

### Evaluation
Submissions are scored on the root mean squared error (RMSE). RMSE is very common and is a suitable general-purpose error metric. Compared to the Mean Absolute Error, RMSE punishes large errors:



In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
from matplotlib import pyplot as plt

In [4]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# content/drive/My Drive/datasource/blackfriday/
Path.cwd()
data_path = 'drive/My Drive/datasource/blackfriday/{file}'

In [7]:
train_file = Path.cwd().joinpath(data_path.format(file='train.csv'))
test_file =  Path.cwd().joinpath(data_path.format(file='test.csv'))

In [8]:
train_file

PosixPath('/content/drive/My Drive/datasource/blackfriday/train.csv')

In [9]:
train_df = pd.read_csv(train_file)

In [10]:
test_df = pd.read_csv(test_file)

In [11]:
test_df_copy = test_df.copy(deep=True)

In [12]:
def extended_describe(dataframe):
    extended_describe_df= dataframe.describe(include='all').T 
    extended_describe_df['null_count']= dataframe.isnull().sum()
    extended_describe_df['unique_count'] = dataframe.apply(lambda x: len(x.unique()))
    return extended_describe_df 

In [13]:
extended_describe(train_df)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,null_count,unique_count
User_ID,550068,NaN,NaN,NaN,1.00303e+06,1727.59,1e+06,1.00152e+06,1.00308e+06,1.00448e+06,1.00604e+06,0,5891
Product_ID,550068,3631,P00265242,1880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3631
Gender,550068,2,M,414259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
Age,550068,7,26-35,219587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7
Occupation,550068,NaN,NaN,NaN,8.07671,6.52266,0,2,7,14,20,0,21
City_Category,550068,3,B,231173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
Stay_In_Current_City_Years,550068,5,1,193821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5
Marital_Status,550068,NaN,NaN,NaN,0.409653,0.49177,0,0,0,1,1,0,2
Product_Category_1,550068,NaN,NaN,NaN,5.40427,3.93621,1,1,5,8,20,0,20
Product_Category_2,376430,NaN,NaN,NaN,9.84233,5.08659,2,5,9,15,18,173638,18


In [14]:
extended_describe(test_df)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,null_count,unique_count
User_ID,233599,NaN,NaN,NaN,1.00303e+06,1726.5,1e+06,1.00153e+06,1.00307e+06,1.00448e+06,1.00604e+06,0,5891
Product_ID,233599,3491,P00265242,829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3491
Gender,233599,2,M,175772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
Age,233599,7,26-35,93428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7
Occupation,233599,NaN,NaN,NaN,8.08541,6.52115,0,2,7,14,20,0,21
City_Category,233599,3,B,98566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
Stay_In_Current_City_Years,233599,5,1,82604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5
Marital_Status,233599,NaN,NaN,NaN,0.41007,0.491847,0,0,0,1,1,0,2
Product_Category_1,233599,NaN,NaN,NaN,5.27654,3.73638,1,1,5,8,18,0,18
Product_Category_2,161255,NaN,NaN,NaN,9.84959,5.09494,2,5,9,15,18,72344,18


In [15]:
train_df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [16]:
from collections import Counter
def detect_outliers(dataset , noutliers , columns):
    outlier_indices = []
    for column in columns:
        # 1st quartile (25%),# 3rd quartile (75%)
        q1 , q3 = np.percentile(dataset[column] , [25 , 75])

        # Interquartile range (IQR)
        iqr = q3 - q1

        # outlier step
        outlier_step = 1.5 * iqr

        lower_bound = q1 - outlier_step
        upper_bound = q3 + outlier_step

        # Determine a list of indices of outliers for feature col
        outlier_list_col = dataset[(dataset[column] < lower_bound) | (
        dataset[column] > upper_bound)].index
        outlier_indices.extend(outlier_list_col)

    outlier_indices = Counter(outlier_indices)
 
    multiple_outliers = list(k for k , v in outlier_indices.items()
                             if v > noutliers)

    return outlier_indices

In [17]:
outliers = detect_outliers(train_df,2,["Purchase"])

In [18]:
remove_idx = list(outliers.keys())

In [19]:
train_df.drop(remove_idx,inplace=True,axis =0)

In [20]:
train_df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [21]:
train_df['Gender_And_Category'] = train_df['Gender'] + train_df['City_Category']
test_df['Gender_And_Category'] = test_df['Gender'] + test_df['City_Category'] 

In [22]:
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
ohe_enc = OneHotCategoricalEncoder(top_categories=None, drop_last=True,variables=['Gender_And_Category' ])
ohe_enc.fit(train_df.drop(['Purchase'],axis=1))
purchase_series = train_df['Purchase']
train_df = ohe_enc.transform(train_df.drop(['Purchase'],axis=1))
test_df = ohe_enc.transform(test_df)
train_df['Purchase'] =purchase_series

In [23]:
def get_mapper(df,feature,target):
    return df.groupby([feature])[target].sum().rank(ascending= False).to_dict()

# gender_mapper = get_mapper(train_df,'Gender','Purchase')
age_mapper = get_mapper(train_df,'Age','Purchase')
# city_category_mapper = get_mapper(train_df,'City_Category','Purchase')
stay_in_mapper = get_mapper(train_df,'Stay_In_Current_City_Years','Purchase')

train_df['Age_TR']= train_df['Age'].map(lambda x:age_mapper.get(x,-1)).astype('int')
# train_df['Gender_TR']= train_df['Gender'].map(lambda x:gender_mapper.get(x,-1)).astype('int')
# train_df['City_Category_TR']= train_df['City_Category'].map(lambda x:city_category_mapper.get(x,-1)).astype('int')
train_df['Stay_In_Current_City_Years_TR']= train_df['Stay_In_Current_City_Years'].map(lambda x:stay_in_mapper.get(x,-1)).astype('int')

test_df['Age_TR']= test_df['Age'].map(lambda x:age_mapper.get(x,-1)).astype('int')
# test_df['Gender_TR']= test_df['Gender'].map(lambda x:gender_mapper.get(x,-1)).astype('int')
# test_df['City_Category_TR']= test_df['City_Category'].map(lambda x:city_category_mapper.get(x,-1)).astype('int')
test_df['Stay_In_Current_City_Years_TR']= test_df['Stay_In_Current_City_Years'].map(lambda x:stay_in_mapper.get(x,-1)).astype('int')

In [24]:
train_df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Gender_And_Category_FA,Gender_And_Category_MC,Gender_And_Category_MA,Gender_And_Category_MB,Gender_And_Category_FB,Purchase,Age_TR,Stay_In_Current_City_Years_TR
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,1,0,0,0,0,8370,7,2
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,1,0,0,0,0,15200,7,2
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1,0,0,0,0,1422,7,2
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1,0,0,0,0,1057,7,2
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,0,1,0,0,0,7969,6,4


In [25]:
#train_count = train_df.groupby(['User_ID'])['User_ID'].count().to_dict()
#test_count = test_df.groupby(['User_ID'])['User_ID'].count().to_dict()

In [26]:
#train_prd_count = train_df.groupby(['Product_ID'])['Product_ID'].count().to_dict()
#test_prd_count =  test_df.groupby(['Product_ID'])['Product_ID'].count().to_dict()

In [27]:
#train_df['User_ID_Count']= train_df['User_ID'].map(lambda x:train_count.get(x,-1)).astype('int')
#test_df['User_ID_Count']= test_df['User_ID'].map(lambda x:test_count.get(x,-1)).astype('int')

In [28]:
#train_df['Product_ID_Count']= train_df['Product_ID'].map(lambda x:train_prd_count.get(x,-1)).astype('int')
#test_df['Product_ID_Count']= test_df['Product_ID'].map(lambda x:test_prd_count.get(x,-1)).astype('int')

In [29]:
train_df['User_ID'] = train_df['User_ID'] - 1000000
test_df['User_ID'] = test_df['User_ID'] - 1000000

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
train_df['Product_ID'] = train_df['Product_ID'].str.replace('P00', '').astype('int')
test_df['Product_ID'] = test_df['Product_ID'].str.replace('P00', '').astype('int')

In [31]:
train_df.fillna(0,inplace=True)
test_df.fillna(0,inplace=True)

In [32]:
category_names = list(train_df.select_dtypes(include=['object']).columns)
category_names

['Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']

In [33]:
drop_cols =['Age', 'Stay_In_Current_City_Years','Gender', 'City_Category']

In [34]:
X = train_df.drop(drop_cols+['Purchase'], axis=1)
y = train_df['Purchase']


In [35]:
X_test = test_df.drop(drop_cols, axis=1)

In [36]:
X.head()

,User_ID,Product_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Gender_And_Category_FA,Gender_And_Category_MC,Gender_And_Category_MA,Gender_And_Category_MB,Gender_And_Category_FB,Age_TR,Stay_In_Current_City_Years_TR
0,1,69042,10,0,3,0.0,0.0,1,0,0,0,0,7,2
1,1,248942,10,0,1,6.0,14.0,1,0,0,0,0,7,2
2,1,87842,10,0,12,0.0,0.0,1,0,0,0,0,7,2
3,1,85442,10,0,12,14.0,0.0,1,0,0,0,0,7,2
4,2,285442,16,0,8,0.0,0.0,0,1,0,0,0,6,4


In [37]:
X_test.head()

,User_ID,Product_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Gender_And_Category_FA,Gender_And_Category_MC,Gender_And_Category_MA,Gender_And_Category_MB,Gender_And_Category_FB,Age_TR,Stay_In_Current_City_Years_TR
0,4,128942,7,1,1,11.0,0.0,0,0,0,1,0,4,2
1,9,113442,17,0,3,5.0,0.0,0,1,0,0,0,1,5
2,10,288442,1,1,5,14.0,0.0,0,0,0,0,1,2,4
3,10,145342,1,1,4,9.0,0.0,0,0,0,0,1,2,4
4,11,53842,1,0,4,5.0,12.0,0,0,0,0,0,1,1


In [38]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)

In [39]:
import xgboost as xgb

In [40]:
from yellowbrick.regressor import ResidualsPlot
from yellowbrick.regressor import PredictionError
from yellowbrick.regressor import AlphaSelection
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error

def residual_plot(model,X_train, y_train,X_validation, y_validation):
    visualizer = ResidualsPlot(model)
    visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
    visualizer.score(X_validation, y_validation)  # Evaluate the model on the test data
    visualizer.show()         

def prediction_error(model,X_train, y_train,X_validation, y_validation):
    visualizer = PredictionError(model)
    visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
    visualizer.score(X_validation, y_validation)  # Evaluate the model on the test data
    visualizer.show()  

def show_learning_curve(model,X_train, y_train,X_validation, y_validation):
    train_sizes, train_scores, validation_scores = learning_curve(
        model, X_train, y_train.values.ravel(), cv=5)
    plot_learning_curve(train_sizes, train_scores, validation_scores)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [41]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)

In [42]:
from xgboost.sklearn import XGBRegressor 

In [43]:
xg_reg = XGBRegressor(objective ='reg:squarederror',learning_rate = 0.1, max_depth = 6, n_estimators = 600) #,tree_method='gpu_hist', gpu_id='0')

In [44]:
Check

NameError: ignored

In [45]:
from sklearn.metrics import mean_squared_error

def rmse(model,X,Y):
    y_ = model.predict(X)
    return mean_squared_error(y_,Y)**0.5

In [ ]:
from xgboost import XGBRegressor
from hyperopt import hp,fmin,tpe
from sklearn.model_selection import cross_val_score, KFold

def objective(params):
    params = {
        'n_estimators' : int(params['n_estimators']),
        'max_depth' : int(params['max_depth']),
        'learning_rate' : float(params['learning_rate'])
    }
    
    clf = XGBRegressor(**params,n_jobs=4)
    score = cross_val_score(clf, X_train,y_train, scoring = rmse, cv=KFold(n_splits=3)).mean()
    print("Parmas {} - {}".format(params,score))
    return score

space = {
    'n_estimators': hp.quniform('n_estimators', 500, 3000, 100),
    'max_depth': hp.quniform('max_depth', 5, 15, 3),
    'learning_rate' : hp.uniform('learning_rate',0.05, 0.15) 
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

In [ ]:
Check

In [46]:
# best = {'n_estimators': 950, 'max_depth': 5, 'learning_rate': 0.11073028178063032}
#best = {'n_estimators': 700, 'max_depth': 8, 'learning_rate': 0.07375044952419624}
best={'learning_rate': 0.11735719234292433, 'max_depth': 9, 'n_estimators': 600}

In [47]:
xg_reg = XGBRegressor(**best)
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.11735719234292433, max_delta_step=0, max_depth=9,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=600, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [ ]:
# prediction_error(xg_reg,X_train, y_train,X_validation, y_validation)

In [48]:
predictions = xg_reg.predict(X_validation)
print(mean_squared_error(y_validation, predictions))
result = np.sqrt(mean_squared_error(y_validation, predictions))
print(result)

6180559.692083187
2486.073146969571


In [ ]:
check

In [49]:
y_test = xg_reg.predict(X_test)
submission = pd.DataFrame()
submission['Purchase'] = y_test
submission['User_ID'] = test_df_copy['User_ID']
submission['Product_ID'] = test_df_copy['Product_ID']

import datetime
FORMAT = '%Y%m%d%H%M%S'
timestamp=datetime.datetime.now().strftime(FORMAT)
filename ="Submission_xgboost_"+timestamp+"_out.csv"
out_path = '/content/drive/My Drive/datasource/blackfriday/{filename}'
out_path = out_path.format(filename= filename)
submission.to_csv(out_path,index=False)

In [ ]:
test_df.shape